# 2098 외판원 순회
220424~220425
- 소감 : 역시나 처음 보는 DP 유형 문제는 혼자서 풀이를 떠올리긴 힘들다. 엄청 길게 고민하진 않고 풀이를 봤는데 이해하기 좀 어려웠다. 두 가지가 어려웠는데, 하나는 점화식을 이해하는 것이었고, 다른 하나는 그 점화식을 코드로 구현하기 위해서 비트마스킹을 활용하는 방법을 이해하는 것이었다. 둘 다 이해하기 어려웠지만 막상 이해하고 나니 비트마스크를 이용해 코드를 구현하는 과정이 상당히 새로워서 재밌었다.
- 아이디어 : 아래의 tsp 함수와 비트마스킹을 활용해 방문상태를 표현하는 것이 핵심이다. 8을 이진법으로 표현하면 1000이며, 이는 `[1, 0, 0, 0]`과 상태라고 생각할 수 있다. 즉, 이진법을 이용해 방문상태를 표현하고 방문 여부 확인 및 방문 상태 반영 등의 작업을 매우 빠르게 처리해줄 수 있다. 이것이 비트마스킹을 활용한 방문상태 표현이고, 문제의 핵심인 점화식은 tsp함수를 보고 이해하기 바란다.
- 알고리즘 분류
    - 다이나믹 프로그래밍
    - 비트마스킹
    - 비트필드를 이용한 다이나믹 프로그래밍
    - 외판원 순회 문제
- 티어 : 골드 1

In [2]:
# 2098 외판원 순회
import sys
input = sys.stdin.readline
N = int(input())
adj_matrix = [tuple(map(int, input().split())) for _ in range(N)]
adj_list = [[] for _ in range(N)]
for a, row in enumerate(adj_matrix):
    for b, cost in enumerate(row):
        if cost != 0: # 갈 수 있는 경우만
            adj_list[a].append((b, cost))

# 방문상태를 표시해주는 binary 값인 status는 오른쪽부터 1번 노드로 친다.
def visited(node, status):
    """방문상태가 status일 때 node의 방문여부 리턴"""
    return (status >> node)%2

def visit(node, status):
    """방문상태가 status일 때 node를 방문한 처리한 new_status 리턴"""
    return status + (0b1<<node)

start = 0 # 비트마스킹 때문에 0으로 고정해야 함
dp = [[float('inf')]*2**N for _ in range(N)] # 아래 tsp 함수의 정보 저장
all_visited = 2**N - 1
def tsp(now_node=start, status=visit(start,0)):
    """방문상태가 status일 때 now_node에서 출발해 start로 돌아오는 최소 비용 리턴"""
    if dp[now_node][status] == float('inf'):
        if status == all_visited: # 전부 다 돈 경우
            if adj_matrix[now_node][start]: # start로 가는 길이 있으면
                dp[now_node][status] = adj_matrix[now_node][start]
            else:
                dp[now_node][status] = 10**8 # 길 없다고 표시
        else:
            candi_costs = [10**8]
            for adj_node, adj_cost in adj_list[now_node]:
                if adj_node != start and not visited(adj_node, status):
                    new_status = visit(adj_node, status)
                    new_cost = adj_cost + tsp(adj_node, new_status)
                    candi_costs.append(new_cost)
            dp[now_node][status] = min(candi_costs)
    return dp[now_node][status]

print(tsp())

In [ ]:
# 2098 외판원 순회 (메모리 적게 먹는 거)
import sys
input = sys.stdin.readline
N = int(input())
adj_matrix = [tuple(map(int, input().split())) for _ in range(N)]
adj_list = [[] for _ in range(N)]
for a, row in enumerate(adj_matrix):
    for b, cost in enumerate(row):
        if cost != 0: # 갈 수 있는 경우만
            adj_list[a].append((b, cost))

# 방문상태를 표시해주는 binary 값인 status는 오른쪽부터 1번 노드로 친다.
def visited(node, status):
    """방문상태가 status일 때 node의 방문여부 리턴"""
    return (status >> node-1)%2

def visit(node, status):
    """방문상태가 status일 때 node를 방문한 처리한 new_status 리턴"""
    return status + (0b1<<(node-1))

start = 0 # 비트마스킹 때문에 0으로 고정해야 함
dp = [[float('inf')]*2**(N-1) for _ in range(N-1)] # 아래 tsp 함수의 정보 저장(0(start)에서 출발하는 경우는 빼도 돼서 N-1로)
all_visited = 2**(N-1) - 1
def tsp(now_node, status=0):
    """방문상태가 status일 때 now_node에서 출발해 start로 돌아오는 최소 비용 리턴"""
    if dp[now_node-1][status] == float('inf'):
        if status == all_visited: # 전부 다 돈 경우
            if adj_matrix[now_node][start]: # start로 가는 길이 있으면
                dp[now_node-1][status] = adj_matrix[now_node][start]
            else:
                dp[now_node-1][status] = 10**8 # 길 없다고 표시해주는 게 나을듯
        else:
            candi_costs = [10**8]
            for adj_node, adj_cost in adj_list[now_node]:
                if adj_node != start and not visited(adj_node, status):
                    new_status = visit(adj_node, status)
                    new_cost = adj_cost + tsp(adj_node, new_status)
                    candi_costs.append(new_cost)
            dp[now_node-1][status] = min(candi_costs)
    return dp[now_node-1][status]

final_costs = []
for node in range(1, N):
    if adj_matrix[start][node]: # start에서 node로 가는 길이 있다면
        status = visit(node, 0)
        final_costs.append(adj_matrix[start][node] + tsp(node, status))

print(min(final_costs))

In [81]:
adj_list, adj_matrix, N

([[(1, 10), (2, 15), (3, 20)],
  [(0, 5), (2, 9), (3, 10)],
  [(0, 6), (1, 13), (3, 12)],
  [(0, 8), (1, 8), (2, 9)]],
 [(0, 10, 15, 20), (5, 0, 9, 10), (6, 13, 0, 12), (8, 8, 9, 0)],
 4)

In [83]:
# 방문상태를 표시해주는 binary 값인 status는 오른쪽부터 1번 노드로 친다.
def visited(node, status):
    """방문상태가 status일 때 node의 방문여부 리턴"""
    return (status >> node)%2

def visit(node, status):
    """방문상태가 status일 때 node를 방문한 처리한 new_status 리턴"""
    return status + (0b1<<(node))

start = 0 # 비트마스킹 때문에 0으로 고정해야 함
dp = [[float('inf')]*2**N for _ in range(N)] # 아래 tsp 함수의 정보 저장
all_visited = 2**N - 1
def tsp(now_node=start, status=visit(start,0)):
    """방문상태가 status일 때 now_node에서 출발해 start로 돌아오는 최소 비용 리턴"""
    if dp[now_node][status] == float('inf'):
        if status == all_visited: # 전부 다 돈 경우
            if adj_matrix[now_node][start]: # start로 가는 길이 있으면
                dp[now_node][status] = adj_matrix[now_node][start]
            else:
                dp[now_node][status] = 10**8 # 길 없다고 표시해주는 게 나을듯
        else:
            candi_costs = [10**8]
            for adj_node, adj_cost in adj_list[now_node]:
                if adj_node != start and not visited(adj_node, status):
                    new_status = visit(adj_node, status)
                    new_cost = adj_cost + tsp(adj_node, new_status)
                    candi_costs.append(new_cost)
            dp[now_node][status] = min(candi_costs)
    return dp[now_node][status]

print(tsp())

35


In [76]:
# 방문상태를 표시해주는 binary 값인 status는 오른쪽부터 1번 노드로 친다.
def visited(node, status):
    """방문상태가 status일 때 node의 방문여부 리턴"""
    return (status >> node-1)%2

def visit(node, status):
    """방문상태가 status일 때 node를 방문한 처리한 new_status 리턴"""
    return status + (0b1<<(node-1))

start = 0 # 비트마스킹 때문에 0으로 고정해야 함
dp = [[float('inf')]*2**(N-1) for _ in range(N-1)] # 아래 tsp 함수의 정보 저장(0(start)에서 출발하는 경우는 빼도 돼서 N-1로)
all_visited = 2**(N-1) - 1
def tsp(now_node, status=0):
    """방문상태가 status일 때 now_node에서 출발해 start로 돌아오는 최소 비용 리턴"""
    if dp[now_node-1][status] == float('inf'):
        if status == all_visited: # 전부 다 돈 경우
            if adj_matrix[now_node][start]: # start로 가는 길이 있으면
                dp[now_node-1][status] = adj_matrix[now_node][start]
            else:
                dp[now_node-1][status] = 10**8 # 길 없다고 표시해주는 게 나을듯
        else:
            candi_costs = [10**8]
            for adj_node, adj_cost in adj_list[now_node]:
                if adj_node != start and not visited(adj_node, status):
                    new_status = visit(adj_node, status)
                    new_cost = adj_cost + tsp(adj_node, new_status)
                    candi_costs.append(new_cost)
            dp[now_node-1][status] = min(candi_costs)
    return dp[now_node-1][status]

final_costs = []
for node in range(1, N):
    if adj_matrix[start][node]: # start에서 node로 가는 길이 있다면
        status = visit(node, 0)
        final_costs.append(adj_matrix[start][node] + tsp(node, status))

print(min(final_costs))

35


In [74]:
for i in range(8):
    print(bin(i)[2:].zfill(3), end=' ')

000 001 010 011 100 101 110 111 

In [77]:
dp, final_costs

([[inf, 25, inf, 18, inf, 15, inf, 5],
  [inf, inf, 25, 20, inf, inf, 18, 6],
  [inf, inf, inf, inf, 23, 15, 13, 8]],
 [35, 40, 43])

In [52]:
# 방문상태를 표시해주는 binary 값인 status는 오른쪽부터 1번 노드로 친다.
def visited(node, status):
    """방문상태가 status일 때 node의 방문여부 리턴"""
    return (status >> node-1)%2

def visit(node, status):
    """방문상태가 status일 때 node를 방문한 처리한 new_status 리턴"""
    return (status + (0b1<<(node-1)))

start = 0 # 비트마스킹 때문에 0으로 고정해야 함
dp = [[float('inf')]*2**(N-1) for _ in range(N-1)] # 아래 tsp 함수의 정보 저장(0(start)에서 출발하는 경우는 빼도 돼서 N-1로)
all_visited = 2**(N-1) - 1
def tsp(now_node=start, status=0):
    """방문상태가 status일 때 now_node에서 출발해 start로 돌아오는 최소 비용 리턴"""
    if dp[now_node-1][status] == float('inf'):
        if status == all_visited: # 전부 다 돈 경우
            if adj_matrix[now_node][start]: # start로 가는 길이 있으면
                dp[now_node-1][status] = adj_matrix[now_node][start]
        else:
            candi_costs = []
            for adj_node, adj_cost in adj_list[now_node]:
                if adj_node != start and not visited(adj_node, status):
                    new_status = visit(adj_node, status)
                    new_cost = adj_cost + tsp(adj_node, new_status)
                    candi_costs.append(new_cost)
            dp[now_node-1][status] = min(candi_costs)
    return dp[now_node-1][status]

print(tsp())

35


In [51]:
dp

[[inf, 25, inf, 18, inf, 15, inf, 5],
 [inf, inf, 25, 20, inf, inf, 18, 6],
 [35, inf, inf, inf, 23, 15, 13, 8]]

In [70]:
min_cost = [float('inf')]
visited = [0]*N
start_city = 0
visited[start_city] = 1
def tsp(now_city=start_city, n=1, until_cost=0):
    """now_city : 현재 위치한 도시, n : 지금까지 방문한 도시 수, until_cost : 지금까지 든 비용"""
    if min_cost[0] <= until_cost:
        return
    if n == N: # N개의 도시를 모두 방문했고 출발했던 도시로 돌아올 수 있으면
        final_cost = until_cost + adj_matrix[now_city][start_city]
        if until_cost < final_cost < min_cost[0]:
            min_cost[0] = final_cost
        return
    for adj_city, adj_cost in adj_list[now_city]:
        if not visited[adj_city]:
            visited[adj_city] = 1
            tsp(adj_city, n+1, until_cost+adj_cost)
            visited[adj_city] = 0

tsp()
print(min_cost[0])

35


In [47]:
bin(8 + (0b1 << (2-1)))

'0b1010'

In [44]:
(8 >> 3)

1

In [42]:
visited(1, 8)

0

In [20]:
bin(0b111001 ^ 0b1001)

'0b110000'

In [33]:
st = 0b1101111
bin(0b1101111 >> (5-1))

'0b110'

In [35]:
0b111111

63

In [38]:
bin(0b110001 + (0b1<<(3-1)))

'0b110101'

In [53]:
bin(0b1<<3)

'0b1000'

In [55]:
10**8

100000000